In [1]:
import numpy as np

import asyncio
from serial_interface.serial_interface import TwidSerialInterfaceProtocol, TelemetryFrame, run_test, ControlType, start_protocol

#control parameter fitting
import scipy.optimize
import scipy.signal

import matplotlib.pyplot as plt
import pandas

import os

###SERIAL CONFIGURATION for esp32
SERIAL_PORT = 'COM9'
SERIAL_BAUD_RATE = 1000000

DATA_DIR_PATH = os.path.join(os.getcwd(),'data')

twid = await start_protocol(asyncio.get_event_loop(), SERIAL_PORT, SERIAL_BAUD_RATE)

port opened SerialTransport(<_WindowsSelectorEventLoop running=True closed=False debug=False>, <serial_interface.serial_interface.TwidSerialInterfaceProtocol object at 0x0000029B9C51E870>, Serial<id=0x29b9cda1990, open=True>(port='COM9', baudrate=1000000, bytesize=8, parity='N', stopbits=1, timeout=0, xonxoff=False, rtscts=False, dsrdtr=False))


In [2]:
#MOTOR EMPIRCAL PARAMS

#voltage supply
PWM_AMPLITUDE = 12
DC_STEPS = [128, 256, 512, 1024]


#second order model
def so_model(x,K,wn,zeta):
    return scipy.signal.lti([K],[1, 2*wn*zeta, wn**2]).step(T=x)[1]

In [18]:
#reset
await twid.esp32_reboot()
twid.clear_frames()
await twid.update_telem_sample_rate(5)
await twid.update_control_type(ControlType.NO_CTRL)
await twid.wait_for_param('control_type', ControlType.NO_CTRL)
await twid.update_dutycycle(0)
await twid.wait_for_param('pwm_duty_cycle',0)

step_input = 512
#apply step to 512/1024
await twid.update_dutycycle(step_input)
#collect all data for next 5 seconds
dataseries = await twid.collect_telem(5)
print(dataseries)
#stop motor
await twid.motor_stop()

#save as a nice csv using pandas
dataseries.to_csv(os.path.join(DATA_DIR_PATH,f'current_step_response_{step_input}.csv'), index=False)

#collect info from start of step only
ts = dataseries['timestamp_ms'][dataseries['pwm_duty_cycle'] >= step_input]
current = dataseries['filtered_current'][dataseries['pwm_duty_cycle'] >= step_input]
vel = dataseries['filtered_velocity'][dataseries['pwm_duty_cycle'] >= step_input]

#apply curve fitting to get second order params using scipy
[K, wn, zeta],_ = scipy.optimize.curve_fit(so_model, ts.to_numpy(), current.to_numpy())
output_str = f'Step response estimated second order parameters:\n\
    K:{K}\twn:{wn}\tzeta:{zeta}\n'
print(output_str)

[K, wn, zeta],_ = scipy.optimize.curve_fit(so_model, ts.to_numpy(), vel.to_numpy())
output_str = f'Step response estimated second order parameters:\n\
    K:{K}\twn:{wn}\tzeta:{zeta}\n'
print(output_str)

     timestamp_ms  loop_dt  control_dt  read_dt  pwm_duty_cycle  \
0           406.0   1011.0        16.0     57.0           512.0   
1           411.0   1008.0        16.0     59.0           512.0   
2           416.0    994.0        24.0     65.0           512.0   
3           421.0    994.0        25.0     65.0           512.0   
4           426.0   1012.0        16.0     57.0           512.0   
..            ...      ...         ...      ...             ...   
152        1166.0    994.0        24.0     65.0           512.0   
153        1171.0   1016.0        40.0     93.0           512.0   
154        1176.0    994.0        24.0     65.0           512.0   
155        1181.0   1011.0        17.0     57.0           512.0   
156        1186.0    994.0        24.0     65.0           512.0   

     pwm_frequency  position  velocity  filtered_velocity    current  ...  \
0             -0.0       0.0       0.0                0.0 -18.382353  ...   
1             -0.0       0.0       0.0   

g:\Apps\.python\Lib\site-packages\scipy\signal\_filter_design.py:1746: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "
